<a href="https://colab.research.google.com/github/codeguima/data-science/blob/jhonnycodedev/Lista%20NPS%20Positivo_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [2]:
from google.colab import files
uploaded = files.upload()


Saving Lista NPS Positivo_V4 (1).xlsx to Lista NPS Positivo_V4 (1).xlsx


In [5]:
import io

# Substitua 'nome_do_arquivo.xlsx' pelo nome do seu arquivo
df = pd.read_excel(io.BytesIO(uploaded['Lista NPS Positivo_V4 (1).xlsx']), engine='openpyxl')


In [ ]:
# Verificando as primeiras linhas
#print(df.head())

# Resumo estatístico
#print(df.describe())

# Informações sobre o DataFrame
#print(df.info())


In [6]:
# Função para classificar as notas
df['target'] = df['nota'].apply(lambda x: 'Promotor' if x >= 9 else ('Neutro' if x >= 7 else 'Detrator'))


In [ ]:
#print(df.groupby('target').count())

In [26]:

# Filtrar para "Grupo de Produto" == "Grupo 2"
grupo_2_df = df[(df['Grupo de Produto'] == 'Grupo 2') & (df['mercado'] == 'BRASIL')]


# Definir as regiões com base nos estados
regioes = {
    "Norte": ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
    "Nordeste": ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    "Centro-Oeste": ['DF', 'GO', 'MT', 'MS'],
    "Sul": ['PR', 'RS', 'SC'],
    "Suldeste": ['ES', 'MG', 'RJ', 'SP']
}

# Função para definir a região com base no estado
def obter_regiao(estado):
    for regiao, estados in regioes.items():
        if estado in estados:
            return regiao
    return 'Outros(não identificado)'  # Para estados fora das regiões definidas



In [27]:
# Crie a coluna 'regiao' se ainda não tiver feito
grupo_2_df['regiao'] = grupo_2_df['estado'].apply(obter_regiao)

# Crie um dicionário para armazenar os DataFrames
dfs_por_regiao = {}

# Use o groupby para dividir o DataFrame por 'regiao'
for regiao, df_regiao in grupo_2_df.groupby('regiao'):
    dfs_por_regiao[regiao] = df_regiao


<ipython-input-27-40cb2707167c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_2_df['regiao'] = grupo_2_df['estado'].apply(obter_regiao)


In [28]:
df_norte = dfs_por_regiao['Norte']
df_sul = dfs_por_regiao['Sul']
df_centro_oeste = dfs_por_regiao['Centro-Oeste']
df_nordeste = dfs_por_regiao['Nordeste']
df_sudeste = dfs_por_regiao['Suldeste']


In [38]:
# Garantir que a coluna 'data_resposta' está no formato datetime no df_norte
df_norte['data_resposta'] = pd.to_datetime(df_norte['data_resposta'], dayfirst=True)

# Adicionar coluna 'safra' (apenas o ano da data) no df_norte
df_filter = df_norte['safra'] = df_norte['data_resposta'].dt.year



In [60]:
# Volumetria total por safra
df_dg = volumetria_safra = df_norte.groupby('safra').size().reset_index(name='quantidade')
volumetria_safra['percentual'] = volumetria_safra['quantidade'] / volumetria_safra['quantidade'].sum() * 100




In [55]:
# Volumetria por período de pesquisa e safra
volumetria_periodo = df_norte.groupby(['Periodo de Pesquisa', 'safra']).size().reset_index(name='quantidade')

In [57]:
# Calcular o percentual com transform
volumetria_periodo['percentual'] = volumetria_periodo.groupby('Periodo de Pesquisa')['quantidade'].transform(lambda x: x / x.sum() * 100)


In [58]:

# Soma total de safras (verificação)
total_safras = volumetria_safra['quantidade'].sum()
verificacao_total = len(df_norte) == total_safras

In [59]:
# Resultados
print("Volumetria por Safra (Total):\n", volumetria_safra)
print("\nVolumetria por Período de Pesquisa:\n", volumetria_periodo)
print(f"\nVerificação Total: {'Correto' if verificacao_total else 'Incorreto'}")

Volumetria por Safra (Total):
    safra  quantidade  percentual
0   2022          25   14.880952
1   2023          81   48.214286
2   2024          62   36.904762

Volumetria por Período de Pesquisa:
    Periodo de Pesquisa  safra  quantidade  percentual
0            12 a 18 M   2022           5   12.820513
1            12 a 18 M   2023          20   51.282051
2            12 a 18 M   2024          14   35.897436
3            18 a 30 M   2022           1    2.702703
4            18 a 30 M   2023          29   78.378378
5            18 a 30 M   2024           7   18.918919
6              3 a 6 M   2022           9   18.367347
7              3 a 6 M   2023          16   32.653061
8              3 a 6 M   2024          24   48.979592
9             6 a 12 M   2022          10   23.255814
10            6 a 12 M   2023          16   37.209302
11            6 a 12 M   2024          17   39.534884

Verificação Total: Correto


In [61]:
# Função para classificar as notas
df_norte['target'] = df_norte['nota'].apply(lambda x: 'Promotor' if x >= 9 else ('Neutro' if x >= 7 else 'Detrator'))

# Verifique se 'safra' foi criada antes, se não, crie-a:
df_norte['data_resposta'] = pd.to_datetime(df_norte['data_resposta'], dayfirst=True)
df_norte['safra'] = df_norte['data_resposta'].dt.year

# Calcular volumetria por safra e target
volumetria_safra_target = df_norte.groupby(['safra', 'target']).size().reset_index(name='quantidade')

# Calcular percentuais dentro de cada safra
volumetria_safra_target['percentual'] = volumetria_safra_target.groupby('safra')['quantidade'].transform(lambda x: x / x.sum() * 100)

# Resultado
print(volumetria_safra_target)


   safra    target  quantidade  percentual
0   2022  Detrator           2    8.000000
1   2022    Neutro           5   20.000000
2   2022  Promotor          18   72.000000
3   2023  Detrator           7    8.641975
4   2023    Neutro          16   19.753086
5   2023  Promotor          58   71.604938
6   2024  Detrator           7   11.290323
7   2024    Neutro           7   11.290323
8   2024  Promotor          48   77.419355
